## Методы сбора и обработки данных из сети Интернет

### Задание 2: скраппинг данных с интернет ресурсов

In [346]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import time
from lxml import html
import random
import numpy as np
import re

### HH.ru

In [404]:
position = 'Data scientist'
main_link = 'https://hh.ru'
start_link = 'https://hh.ru/search/vacancy?text=' + position.replace(' ','+') + '&page='
headers = {'User-agent': 'Chrome/76.0.3809.132'}
var_page = '0'

# Определяет кол-во страниц для парсинга
def find_max_pages_hh(start_link, var_page):
    html_page = requests.get(start_link + var_page, headers=headers).text
    parsed_html = bs(html_page, 'html.parser')
    page_bar = parsed_html.findChildren('a',{'class':'bloko-button HH-Pager-Control'})
    max_page_num = int(page_bar[len(page_bar)-1]['data-page'])
    return max_page_num  


max_page = find_max_pages_hh(start_link, var_page)

In [405]:
data_hh = []

for i in range(max_page):
    current_html = requests.get(main_link + '/search/vacancy?L_is_autosearch=false&area=1&clusters=true&enable_snippets=true&text=Data+scientist&page=' + str(i), headers=headers).text
    parsed_html = bs(current_html, 'html.parser')
    vacancy_list = parsed_html.findAll('div',{'class':'vacancy-serp-item'})
    print(f'Обработка {i+1} страницы из {max_page + 1}')

        # Парсит блоки на текущей странице
    for vacancy in vacancy_list:
        # print('.', end='')
        vacancy_number = int(vacancy.find('a',{'class':'bloko-link HH-LinkModifier'})['data-position']) # № Вакансии на сайте
        vacancy_name = vacancy.find('a',{'class':'bloko-link HH-LinkModifier'}).getText() # Наименование вакансии
        try:
            firm_name = vacancy.find('a',{'class':'bloko-link bloko-link_secondary'}).getText() # Фирма
        except:
            firm_name = None
        vacancy_link = vacancy.find('a',{'class':'bloko-link HH-LinkModifier'})['href'] # Ссылка
        salary_min = None # З.п (мин)
        salary_max = None # З.п (макс)
        salary_block = vacancy.find('div',{'class':'vacancy-serp-item__sidebar'}).getText().replace('\xa0',' ')
        if salary_block.find('от') != -1:
            salary_min = int(salary_block.split()[1] + salary_block.split()[2])
        elif salary_block.find('до') != -1:
            salary_max = int(salary_block.split()[1] + salary_block.split()[2])
        elif salary_block.find('-') != -1:
            salary_min = int((salary_block.split('-')[0]).replace(' ',''))
            salary_max = int((salary_block.split('-')[1]).split(' ')[0] + (salary_block.split('-')[1]).split(' ')[1])

        publish_date = vacancy.find('span',{'class': 'vacancy-serp-item__publication-date'}).getText().replace('\xa0',' ') # Добавляет "сырую дату актуальности"
        tasks_shrt = vacancy.find('div',{'class': 'g-user-content'}).contents[0].getText() # Краткое описание деятельности
        prereq_shrt = vacancy.find('div',{'class': 'g-user-content'}).contents[1].getText() # Краткое описание требований
        city = vacancy.find('span',{'class': 'vacancy-serp-item__meta-info'}).contents[0].replace(',','') # Город
        try:
            metro_station = vacancy.find('span',{'class': 'vacancy-serp-item__meta-info'}).contents[1].getText() # Станция метро
        except:
            metro_station = None    

        data_hh.append({
                    'vacancy_name': vacancy_name,
                    'firm_name': firm_name,
                    'vacancy_link': vacancy_link,
                    'salary_min': salary_min,
                    'salary_max': salary_max,
                    'publish_date': publish_date,
                    'tasks_shrt': tasks_shrt,
                    'prereq_shrt': prereq_shrt,
                    'city': city,
                    'metro_station': metro_station,
                    'site': 'hh.ru'

            })

    

    time.sleep(2 + np.random.rand())

Обработка 1 страницы из 21
Обработка 2 страницы из 21
Обработка 3 страницы из 21
Обработка 4 страницы из 21
Обработка 5 страницы из 21
Обработка 6 страницы из 21
Обработка 7 страницы из 21
Обработка 8 страницы из 21
Обработка 9 страницы из 21
Обработка 10 страницы из 21
Обработка 11 страницы из 21
Обработка 12 страницы из 21
Обработка 13 страницы из 21
Обработка 14 страницы из 21
Обработка 15 страницы из 21
Обработка 16 страницы из 21
Обработка 17 страницы из 21
Обработка 18 страницы из 21
Обработка 19 страницы из 21
Обработка 20 страницы из 21


In [406]:
len(data_hh)

213

In [407]:
ordered_cols = ['firm_name', 'vacancy_name', 'salary_min', 'salary_max',
                'city', 'metro_station', 'prereq_shrt', 'tasks_shrt', 'publish_date', 'vacancy_link', 'site']
df_hh = pd.DataFrame.from_dict(data_hh)
df_hh = df_hh[ordered_cols]

In [408]:
df_hh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 11 columns):
firm_name        212 non-null object
vacancy_name     213 non-null object
salary_min       41 non-null float64
salary_max       22 non-null float64
city             213 non-null object
metro_station    110 non-null object
prereq_shrt      213 non-null object
tasks_shrt       213 non-null object
publish_date     213 non-null object
vacancy_link     213 non-null object
site             213 non-null object
dtypes: float64(2), object(9)
memory usage: 18.4+ KB


In [409]:
df_hh.head()

,firm_name,vacancy_name,salary_min,salary_max,city,metro_station,prereq_shrt,tasks_shrt,publish_date,vacancy_link,site
0,HeadHunter::Технический департамент,Big Data Engineer,250000.0,NaN,Москва,Алексеевская,Уверенные знания принципов работы баз данных и...,"Создание ETL-процессов, используя внутренние и...",23 сентября,https://hh.ru/vacancy/32182817?query=Data%20sc...,hh.ru
1,HeadHunter::Технический департамент,Lead devops engineer (Hadoop),300000.0,NaN,Москва,Алексеевская,Опыт работы с распределенными системами хранен...,Обеспечивать непрерывную и корректную работу к...,23 сентября,https://hh.ru/vacancy/33575266?query=Data%20sc...,hh.ru
2,ООО Содружество,Data scientist,NaN,200000.0,Москва,None,...задач различных отраслей экономики в целях ...,Взаимодействие с подрядчиками по проекту скори...,23 сентября,https://hh.ru/vacancy/33713623?query=Data%20sc...,hh.ru
3,ООО Простое решение,Data Scientist,120000.0,180000.0,Москва,Марьина Роща,Практический опыт анализа данных на R/Python. ...,Разработка моделей оценки эффективности сервис...,23 сентября,https://hh.ru/vacancy/33671339?query=Data%20sc...,hh.ru
4,Связной,Главный аналитик-математик (Data Analyst / Dat...,180000.0,230000.0,Москва,Автозаводская,Хорошее владение языком R (предпочтительно) ил...,Исследование данных методами математической ст...,23 сентября,https://hh.ru/vacancy/30250856?query=Data%20sc...,hh.ru


### Superjob.ru

In [357]:
position = 'python'
main_link_sj = 'https://www.superjob.ru'
search_sj = '/vacancy/search/?keywords=' + position.replace(' ','-')
headers = {'User-agent': 'Chrome/76.0.3809.132'}

html = requests.get(main_link_sj + search_sj, headers=headers).text
parsed_html = bs(html, 'html.parser')

In [358]:
def find_last_page(parsed_html):
    page_bar = parsed_html.find('a', {'class': 'f-test-button-1'})
    if not page_bar:
            last_page = 1
    else:
        page_bar = page_bar.findParent()
        last_page = int(page_bar.find_all('a')[-2].getText())
    return last_page

max_page = find_last_page(parsed_html)

In [397]:
data_sj = []

for i in range(max_page):
    current_html = requests.get(main_link_sj + search_sj + '&page=' + str(i + 1), headers=headers).text
    parsed_html = bs(current_html, 'html.parser')    
    vacancy_list = parsed_html.findAll('div', {'class': 'f-test-vacancy-item'})
    print(f'Обработка {i+1} страницы из {max_page}')

        # Парсит блоки на текущей странице
    for vacancy in vacancy_list:
        # print('.',end='')
        vacancy_name = vacancy.find('a').getText()
        firm_name = vacancy.findAll('a')[1].getText()
        
        if vacancy_name == '':
            vacancy_name = vacancy.findAll('a')[1].getText()
            firm_name = vacancy.findAll('a')[2].getText()
            
        vacancy_link = main_link_sj + vacancy.find('a')['href']
        
        location = vacancy.find('span', {'class': 'f-test-text-company-item-location'}).findChildren()[1].getText().split(',')
        if not location:
            location = None
        city = location[0]
        metro_station = None
        if len(location) > 1:
            metro_tmp = location[1:]
            metro_station = ''
            for i in range(len(metro_tmp)):
                metro_station += metro_tmp[i] + ' '
                
                
        salary_block = vacancy.find('span', {'class': 'f-test-text-company-item-salary'}).findChildren()
        
        if len(salary_block) == 4:
            salary_min = salary_block[0].getText().replace('\xa0', '')
            salary_max = salary_block[2].getText().replace('\xa0', '')
        elif len(salary_block) == 2:
            search_condition = vacancy.find('span', {'class': 'f-test-text-company-item-salary'}).getText().replace('\xa0', ' ').split(' ')
            if search_condition[0] == 'от':
                salary_min = search_condition[1] + search_condition[2]
                salary_max = None
            elif search_condition[0] == 'до':
                salary_min = None
                salary_max = search_condition[1] + search_condition[2]
            elif search_condition[0] == 'По':
                salary_min = None
                salary_max = None
            else:
                salary_min = search_condition[0] + search_condition[1]
                salary_max = salary_min
        publish_date = vacancy.find('span', {'class': 'f-test-text-company-item-location'}).findChildren()[0].getText()
        decribe_block = vacancy.find('div', {'class': '_1tH7S _10Aay _3C76h _3achh _3ofxL _2_FIo'}).getText()
        desc = re.split('\Должностные обязанности: |Требования: ', decribe_block)
        try:
            tasks_shrt = desc[1]
        except:
            tasks_shrt = None
        try:
            prereq_shrt = desc[2]
        except:
            prereq_shrt = None
            
            
            
        
        
        
        data_sj.append({
                    'vacancy_name': vacancy_name,
                    'firm_name': firm_name,
                    'vacancy_link': vacancy_link,
                    'salary_min': salary_min,
                    'salary_max': salary_max,
                    'publish_date': publish_date,
                    'tasks_shrt': tasks_shrt,
                    'prereq_shrt': prereq_shrt,                    
                    'city': city,
                    'metro_station': metro_station,
                    'site': 'superjob.ru'

            })
 

    time.sleep(2 + np.random.rand())

Обработка 1 страницы из 2
Обработка 2 страницы из 2


In [400]:
ordered_cols = ['firm_name', 'vacancy_name', 'salary_min', 'salary_max',
                'city', 'metro_station', 'prereq_shrt', 'tasks_shrt', 'publish_date', 'vacancy_link', 'site']
df_sj = pd.DataFrame.from_dict(data_sj)
df_sj = df_sj[ordered_cols]

In [401]:
df_sj

,firm_name,vacancy_name,salary_min,salary_max,city,metro_station,prereq_shrt,tasks_shrt,publish_date,vacancy_link,site
0,Business connection,"Программист С++, Golang, PHP, Python, DevOPS (...",100000,250000,Москва,None,Знание языка программирования C++ на продвинут...,О компании. Написание с нуля без использования...,13:18,https://www.superjob.ru/clients/business-conne...,superjob.ru
1,Performance Group,Web-аналитик (Python),120000,150000,Москва,Павелецкая Пролетарская,Что мы хотим видеть в тебе. Понимание работы к...,Построить и поддерживать систему сквозной анал...,вчера,https://www.superjob.ru/vakansii/web-analitik-...,superjob.ru
2,НТИМИ,Программист на python,120000,150000,Москва,None,"Опыт программирования на Python от 2 лет, базо...",Вам предстоит разрабатывать автоматизированную...,20 сентября,https://www.superjob.ru/vakansii/programmist-n...,superjob.ru
3,ВНИИМС,Аналитик (инженер-математик),80000,None,Москва,Озёрная,"Высшее образование в области математики, техни...",Разработка прогнозных моделей в части оценки п...,21:14,https://www.superjob.ru/vakansii/analitik-3256...,superjob.ru
4,Тинькофф,Специалист Отдела Операционного планирования и...,50000,None,Москва,None,"Логическое, абстрактное мышление, умение сопос...",Сбор и анализ информации по работе подразделен...,20:35,https://www.superjob.ru/clients/tinkoff-160502...,superjob.ru
5,Концерн Автоматика,Главный специалист отдела инновационных разраб...,140000,140000,Москва,Владыкино,"Уверенные знания и опыт работы с Python, админ...","Разработка приложений, интеграционных сервисов...",17:48,https://www.superjob.ru/vakansii/glavnyj-speci...,superjob.ru
6,МобильныеТелеСистемы,Старший инженер эксплуатации ТВ платформ,140000,140000,Москва,Пражская,None,Образование высшее в области связи и радиотехн...,15:25,https://www.superjob.ru/clients/mts-81084.html,superjob.ru
7,Правительство Москвы,Тестировщик программного обеспечения,140000,140000,Москва,None,Высшее техническое образование. Знание и опыт ...,Тестирование функционала информационных систем...,14:40,https://www.superjob.ru/clients/pravitelstvo-m...,superjob.ru
8,Правительство Москвы,Главный эксперт аналитики баз данных,140000,140000,Москва,Марьина роща,"Уверенное владение SQL (joinы, вложенные запро...",Сбор большого количества разнородных данных из...,13:40,https://www.superjob.ru/clients/pravitelstvo-m...,superjob.ru
9,МобильныеТелеСистемы,Ведущий инженер Команда МТС ТВ,110000,110000,Москва,None,Образование высшее в области связи и радиотехн...,"Администрирование, конфигурация и поддержка се...",13:15,https://www.superjob.ru/clients/mts-81084.html,superjob.ru


In [410]:
df = df_hh.append(df_sj, ignore_index=True)
df

,firm_name,vacancy_name,salary_min,salary_max,city,metro_station,prereq_shrt,tasks_shrt,publish_date,vacancy_link,site
0,HeadHunter::Технический департамент,Big Data Engineer,250000,NaN,Москва,Алексеевская,Уверенные знания принципов работы баз данных и...,"Создание ETL-процессов, используя внутренние и...",23 сентября,https://hh.ru/vacancy/32182817?query=Data%20sc...,hh.ru
1,HeadHunter::Технический департамент,Lead devops engineer (Hadoop),300000,NaN,Москва,Алексеевская,Опыт работы с распределенными системами хранен...,Обеспечивать непрерывную и корректную работу к...,23 сентября,https://hh.ru/vacancy/33575266?query=Data%20sc...,hh.ru
2,ООО Содружество,Data scientist,NaN,200000,Москва,None,...задач различных отраслей экономики в целях ...,Взаимодействие с подрядчиками по проекту скори...,23 сентября,https://hh.ru/vacancy/33713623?query=Data%20sc...,hh.ru
3,ООО Простое решение,Data Scientist,120000,180000,Москва,Марьина Роща,Практический опыт анализа данных на R/Python. ...,Разработка моделей оценки эффективности сервис...,23 сентября,https://hh.ru/vacancy/33671339?query=Data%20sc...,hh.ru
4,Связной,Главный аналитик-математик (Data Analyst / Dat...,180000,230000,Москва,Автозаводская,Хорошее владение языком R (предпочтительно) ил...,Исследование данных методами математической ст...,23 сентября,https://hh.ru/vacancy/30250856?query=Data%20sc...,hh.ru
5,ООО АлгоМост,Computer Vision / Data scientist CV,100000,200000,Москва,None,"Знание и опыт работы в сфере анализа данных, м...",Построение моделей для вычисления количественн...,23 сентября,https://hh.ru/vacancy/33352742?query=Data%20sc...,hh.ru
6,Nielsen,Data Scientist,NaN,NaN,Москва,Тушинская,"Bachelor’s degree in Mathematics, Economics or...",Work with clients to identify key business iss...,23 сентября,https://hh.ru/vacancy/33712147?query=Data%20sc...,hh.ru
7,ЛОКО-БАНК,Data Scientist,NaN,NaN,Москва,None,Хорошие знания математической статистики и тео...,Участие в генерации гипотез и бизнес идей; оце...,23 сентября,https://hh.ru/vacancy/33478456?query=Data%20sc...,hh.ru
8,АО Инфотех Груп,Data scientist,NaN,NaN,Москва,None,Опыт работы не менее 2 лет в области Data Scie...,Участие во всем цикле работ по созданию и внед...,23 сентября,https://hh.ru/vacancy/33729996?query=Data%20sc...,hh.ru
9,"«Почта России», Офис",Data Scientist,NaN,NaN,Москва,Нагатинская,"Оконченное высшее образование (техническое, ма...",Развитие системы анализа больших данных Блока ...,23 сентября,https://hh.ru/vacancy/32858945?query=Data%20sc...,hh.ru


In [411]:
df.to_csv('job_search.csv',index=None)